In [1]:
from __future__ import print_function
import os
import neat
import json

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode


from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from copy import deepcopy

import time
from datetime import datetime


import gzip
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

In [2]:

USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


In [3]:
device

device(type='cpu')

# Iris Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [4]:
my_random_seed = 42
random.seed(my_random_seed)

In [5]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [6]:
iris = datasets.load_iris()
xs_raw = iris.data[:, :]
scaler = StandardScaler()
scaler.fit(xs_raw)
xs = scaler.transform(xs_raw)
ys = iris.target
ys_onehot = one_hot_encode(ys)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(xs, ys, test_size=45, random_state=42)

In [8]:
# xs = torch.from_numpy(xs).to(device)
# ys = torch.from_numpy(ys).to(device)
X_train = torch.from_numpy(X_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
y_train = torch.from_numpy(y_train).to(device)
y_test = torch.from_numpy(y_test).to(device)

Let's have a look at the data we are working with

In [ ]:
xs[:5]

array([[-0.90068117,  1.01900435, -1.34022653, -1.3154443 ],
       [-1.14301691, -0.13197948, -1.34022653, -1.3154443 ],
       [-1.38535265,  0.32841405, -1.39706395, -1.3154443 ],
       [-1.50652052,  0.09821729, -1.2833891 , -1.3154443 ],
       [-1.02184904,  1.24920112, -1.34022653, -1.3154443 ]])

In [ ]:
ys[:5]

array([0, 0, 0, 0, 0])

In [ ]:
ys_onehot[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `CrossEntropyLoss` from `PyTorch`

In [ ]:
def eval_genomes(genomes, config):
    loss = nn.CrossEntropyLoss()
    loss = loss.to(device)

    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in X_train:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(y_train)))

## Base configuration

We are going to create the base configuration according to an external configuration file. Per experiment, we will adjust this, later, but this defines the defaults across all runs.

In [ ]:
config_path = "./config-iris"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)


We also want to put a hard limit on how long this can go on for.

In [ ]:
maxNGenerations = 200

We will create a method to manage the instantiation of a population on the basis of a specific config.

In [ ]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, 
                            xs, 
                            ys, 
                            criterion=nn.CrossEntropyLoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

# Experiment 1: Vary population size

The first experiment is going to examine the difference in run time different population sizes. 

In [ ]:
# epoch_points = [10, 25, 50, 100, 150]
epoch_points = [10]

In [ ]:
base_config.pop_size

50

In [ ]:
saveLocationTemplate = './../../data/experiments/iris/experiment-longepochsttsplit-{}-{}/'

## Start the experiment

In [ ]:
for epochs in epoch_points:
    for iteration_no in range(20):
        my_random_seed += 1
        random.seed(my_random_seed)
        start_time = datetime.now()
        
        print("################################################")
        print("################################################")
        print("Starting epochs {} iteration {}".format(epochs, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
        
        
        config = deepcopy(base_config)
#         config.pop_size = pop_size
        
        saveLocation = saveLocationTemplate.format(epochs, iteration_no)
        
        p = instantiate_population(config, X_train, y_train, saveLocation)
        # Run for up to nGenerations generations.
        winner = p.run(eval_genomes, maxNGenerations, nEpochs = epochs)
        
        g = p.best_genome

        
        end_time = datetime.now()
        
        p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  
        
        winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

        results = []
        for xi, xo in zip(xs, ys):
            output = winner_net.activate(xi)
            results.append([xi, xo, output])

        df = pd.DataFrame(results)
        df.to_csv(os.path.join(saveLocation, 'results.csv'))
        
        ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

        ancestors = {
            k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
        }
        
    
        
        ## Save all of these to disc
        filename = 'fullStatus.xplnt'
        print("Saving checkpoint to {0}".format(filename))

        with gzip.open(os.path.join(saveLocation, filename), 'w', compresslevel=5) as f:
            data = (p, g, ancestry, ancestors, random.getstate())
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
            
            
        with gzip.open(os.path.join(saveLocation, 'train_test_data.pkl'), 'w', compresslevel=5) as f:
            train_Test = (X_train, X_test, y_train, y_test)
            pickle.dump(train_Test, f, protocol=pickle.HIGHEST_PROTOCOL)
#         visualize.create_ancestry_video(p.config, 
#                                         g, 
#                                         ancestry, 
#                                         ancestors, 
#                                         p.reporters.reporters[1], 
#                                         pathname=saveLocation)
        print("################################################")
        print("################################################")
        print("Have finished epochs {} iteration {}".format(epochs, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("The time is {}".format(end_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
    

################################################
################################################
Starting epochs 10 iteration 0
Started at 08/26/2019, 03:06:21
################################################
################################################

 ****** Running generation 0 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7511, grad_fn=<DivBackward0>)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Population's average fitness: 0.84966 stdev: 0.13650
Best fitness: 1.33140 - size: (3, 12) - species 5 - id 30
ending generation %s
Average adjusted fitness: 0.225
Mean genetic distance 3.543, standard deviation 1.020
Population of 49 members in 10 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    10      1.1    0.280     0
     2    0     9      1.1    0.238     0
     3    0     2      0.9    0.169     0
     4    0     2      1.0    0.255     0
     5    0     7      1.3    0.252     0
     6    0     2      0.8    0.142     0
     7    0     4      0.9    0.292     0
     8    0     9      1.0    0.278     0
     9    0     1      0.8    0.133     0
    10    0     3      0.8    0.214     0
Total extinctions: 0
Generation time: 23.911 sec

 ****** Running generation 1 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7511, grad_fn=<DivBackward0>)
Population's average fitn

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7086, grad_fn=<DivBackward0>)
Population's average fitness: 1.11481 stdev: 0.18087
Best fitness: 1.41122 - size: (5, 14) - species 8 - id 208


 SPECIES TOPOLOGY IMPROVEMENT


{'genome': <neat.genome.DefaultGenome object at 0x7f8b0a3f0668>, 'fitness': 1.4112168550491333, 'firstDerivatives': [0.0, 0.0, 0.058147549629211426, 0.0028368234634399414, 0.0, 0.018836379051208496], 'secondDerivatives': [0.0, 0.0, 0.058147549629211426, -0.055310726165771484, -0.0028368234634399414, 0.018836379051208496]}
Key: 208
Fitness: 1.4112168550491333
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.14485633373260498, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=0.7926075458526611, response=1.0, activation=sigmoid, aggregation=sum)
	2 DefaultNodeGene(key=2, bias=-0.007446212228387594, response=1.0, activation=sigmoid, aggregation=sum)
	35 DefaultNodeGene(key=35, bias=-0.56451284885406

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.7104, grad_fn=<DivBackward0>)
Population's average fitness: 1.11996 stdev: 0.19501
Best fitness: 1.41806 - size: (7, 18) - species 8 - id 353


 SPECIES TOPOLOGY IMPROVEMENT


{'genome': <neat.genome.DefaultGenome object at 0x7f8b0a36fa58>, 'fitness': 1.4180628061294556, 'firstDerivatives': [0.0, 0.0, 0.058147549629211426, 0.0028368234634399414, 0.0, 0.018836379051208496, 0.0, 7.975101470947266e-05, 7.224082946777344e-05, 0.00012958049774169922, 0.00656437873840332], 'secondDerivatives': [0.0, 0.0, 0.058147549629211426, -0.055310726165771484, -0.0028368234634399414, 0.018836379051208496, -0.018836379051208496, 7.975101470947266e-05, -7.510185241699219e-06, 5.733966827392578e-05, 0.006434798240661621]}
Key: 353
Fitness: 1.4180628061294556
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.14485633373260498, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=0.7926075458526

...


 ****** Running generation 159 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6078, grad_fn=<DivBackward0>)
Population's average fitness: 1.37026 stdev: 0.21874
Best fitness: 1.64534 - size: (15, 35) - species 8 - id 6193


 SPECIES TOPOLOGY IMPROVEMENT


{'genome': <neat.genome.DefaultGenome object at 0x7f8af92cc278>, 'fitness': 1.645344614982605, 'firstDerivatives': [0.0, 0.0, 0.058147549629211426, 0.0028368234634399414, 0.0, 0.018836379051208496, 0.0, 7.975101470947266e-05, 7.224082946777344e-05, 0.00012958049774169922, 0.00656437873840332, 0.012216806411743164, 0.0, 0.005431532859802246, 0.0, 0.01624596118927002, 0.0, 0.02496922016143799, 0.002549290657043457, 0.006128430366516113, 0.0, 0.007392406463623047, 0.0, 0.0, 0.0, 0.024271368980407715, 0.001916646957397461, 0.0, 0.0026383399963378906, 0.0084608793258667, 0.0, 0.0, 0.005444049835205078, 0.0, 0.003247976303100586, 0.00021922588348388672, 0.0035619735717773438, 0.0, 0.

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6078, grad_fn=<DivBackward0>)
Population's average fitness: 1.40446 stdev: 0.20231
Best fitness: 1.64534 - size: (15, 35) - species 8 - id 6193
ending generation %s
Average adjusted fitness: 0.480
Mean genetic distance 2.355, standard deviation 1.155
Population of 50 members in 5 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1  160    11      1.6    0.506     2
     2  160    10      1.5    0.494     6
     3  160     7      1.4    0.351     9
     4  160    10      1.4    0.433     9
     8  160    12      1.6    0.615     1
Total extinctions: 0
Generation time: 58.689 sec (61.235 average)

 ****** Running generation 161 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6078, grad_fn=<DivBackward0>)
Population's average fitness: 1.41226 stdev: 0.19766
Best fitness: 1.64534 - size: (15, 3

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6078, grad_fn=<DivBackward0>)
Population's average fitness: 1.44155 stdev: 0.19150
Best fitness: 1.64607 - size: (40, 68) - species 1 - id 6624
ending generation %s
Average adjusted fitness: 0.515
Mean genetic distance 2.425, standard deviation 1.118
Population of 50 members in 5 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1  171    10      1.6    0.538     1
     2  171    10      1.5    0.520     0
     3  171     8      1.4    0.384     0
     4  171     9      1.4    0.469     5
     8  171    13      1.6    0.664     0
Total extinctions: 0
Generation time: 65.790 sec (64.600 average)

 ****** Running generation 172 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6078, grad_fn=<DivBackward0>)
Population's average fitness: 1.44227 stdev: 0.15656
Best fitness: 1.64607 - size: (39, 6

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6077, grad_fn=<DivBackward0>)
Population's average fitness: 1.41418 stdev: 0.17386
Best fitness: 1.64894 - size: (39, 67) - species 1 - id 6865
ending generation %s
Average adjusted fitness: 0.491
Mean genetic distance 2.411, standard deviation 1.174
Population of 51 members in 5 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1  177    12      1.6    0.577     1
     2  177    10      1.5    0.451     4
     3  177     8      1.4    0.391     6
     4  177     9      1.4    0.441     5
     8  177    12      1.6    0.593     0
Total extinctions: 0
Generation time: 63.717 sec (64.938 average)

 ****** Running generation 178 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6077, grad_fn=<DivBackward0>)
Population's average fitness: 1.41231 stdev: 0.21042
Best fitness: 1.64894 - size: (39, 6

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6050, grad_fn=<DivBackward0>)
Population's average fitness: 1.44799 stdev: 0.17905
Best fitness: 1.65288 - size: (20, 43) - species 8 - id 7306
ending generation %s
Average adjusted fitness: 0.455
Mean genetic distance 2.476, standard deviation 1.233
Population of 49 members in 5 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1  187    12      1.7    0.552     0
     2  187     9      1.5    0.445     2
     3  187     8      1.4    0.331    16
     4  187     8      1.4    0.389     4
     8  187    12      1.7    0.556     0
Total extinctions: 0
Generation time: 70.995 sec (68.534 average)

 ****** Running generation 188 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6050, grad_fn=<DivBackward0>)
Population's average fitness: 1.41368 stdev: 0.21553
Best fitness: 1.65288 - size: (20, 4